In [ ]:
from torch.utils.data import Dataset , DataLoader
import pandas as pd
from transformers import  GPT2LMHeadModel, GPT2Tokenizer,AdamW
import json
class ReceipeDataset(Dataset):
    def __init__(self,datas, tokenizer):
        self.tokenizer = tokenizer
        # self.ingredients = datas['ingredients'].values
        # self.titles = datas['title'].values
        self.directions = datas['full_recipe'].values
    def __len__(self):
        return len(self.directions)

    def __getitem__(self,index):
        # ingredient = self.ingredients[index]
        direction = self.directions[index]
        # ingredient_token = self.tokenizer.encode(ingredient , max_length = 300 , padding = "max_length" , truncation = True, return_tensors = "pt").reshape(-1)
        direction_token = self.tokenizer.encode(direction , max_length = 200 , padding = "max_length" , truncation = True, return_tensors = "pt").reshape(-1)

        return direction_token




# if __name__ == '__main__':

#     with open('test.json', 'r') as file:
#         data = json.load(file)

#     df = pd.read_csv("test.csv")
#     tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
#     tokenizer.pad_token = tokenizer.eos_token
#     dset = ReceipeDataset(df,tokenizer)
#     title = next(iter(DataLoader(dset , batch_size = 1,shuffle = True)))
#     print(title)

KeyboardInterrupt: 

In [ ]:
# %cd /content
# %ls
from google.colab import drive
drive.mount('/gdrive')

%cd '/gdrive/My Drive/ai-finalproject'

Mounted at /gdrive
/gdrive/My Drive/ai-finalproject


In [ ]:
from transformers import  GPT2LMHeadModel, GPT2Tokenizer,AdamW
import pandas as pd
from torch.utils.data import Dataset , DataLoader
# import pytorch_lightning as pl
from sklearn.model_selection import train_test_split

df = pd.read_csv('data_2/vegan_processed.csv', nrows = 6000*16)
# df = pd.read_csv('vegan_processed.csv')
# df = pd.read_csv('data_2/lacto_ovo_processed.csv', nrows = 6000*16)
# df = pd.read_csv('all_processed.csv', nrows = 6000*16)

gpt2 = GPT2LMHeadModel.from_pretrained("gpt2", cache_dir="./cache/gpt2model")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", cache_dir="./cache/gpt2token")
tokenizer.pad_token = tokenizer.eos_token



# df = pd.read_csv('vegan_processed.csv',index_col = False)
# import csv

x_train , x_test = train_test_split(df, test_size = 0.3 , random_state = 42)

train = ReceipeDataset(x_train,tokenizer)
test = ReceipeDataset(x_test,tokenizer)
val = ReceipeDataset(x_test,tokenizer)

train_loader = DataLoader(train , batch_size = 8 , shuffle = True, pin_memory=True)
test_loader = DataLoader(test , batch_size = 8 , shuffle = True, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch.nn as nn
# from ReceipeDataset import ReceipeDataset
from transformers import AdamW



class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = gpt2

    def forward(self,x):
        # print(x)
        return self.model(x , labels = x)


In [ ]:
import torch
from transformers import Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


num_epochs = 3

learning_rate = 1e-4

model = gpt2.to(device)
model.train()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

n_total_steps = len(train_loader)
scaler = GradScaler()


num_epochs = 4
accumulation_steps = 4



for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}')

    for i, (inputs) in enumerate(progress_bar):
        inputs = inputs.to('cuda')
        with autocast():
            outputs = model(input_ids=inputs, labels=inputs)
            loss = outputs.loss / accumulation_steps

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        running_loss += loss.item() * accumulation_steps
        progress_bar.set_postfix(loss=running_loss / (i + 1))

    scheduler.step()
print('Finished Training')
output_path = 'FinalModel/vegan.pt'
torch.save(model.state_dict(), output_path)
print('save model')



Epoch 4/4: 100%|██████████| 8400/8400 [27:37<00:00,  5.07it/s, loss=0.795]


Finished Training


In [ ]:
print(outputs.logits.size())

torch.Size([16, 300, 50257])


In [ ]:
output_path = 'FinalModel/vegan.pt'
model_path = output_path
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

NameError: name 'GPT2LMHeadModel' is not defined

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# Set the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
raw_text = ["Ingredients: milk. Title:" ,"Ingredients: 1 1/2 c. sugar\n1/2 c. butter\n1 egg\n1 c. buttermilk\n2 c. flour\n1/2 tsp. salt\n1 tsp. soda\n1 c. buttermilk\n2 c. rhubarb, finely cut\n1 tsp. vanilla.\nTitle:" , "Ingredients: 1 can Spanish rice 1 can corned beef 1 diced onion 1/2 c. grated Cheddar cheese. Title:",
            "Ingredients: 3 1/2 c. flour\n3 c. sugar\n2 tsp. baking soda\n1 1/2 tsp. salt\n1 tsp. cinnamon\n1 tsp. nutmeg\n1 c. oil\n2/3 c. water\n4 eggs\n2 c. canned pumpkin.\nTitle: Pumpkin Bread\nReceipe: "]
output_text = []
raw_text = ["Use:\ncarrots, cut in diagonal large onion, sliced in rings\ntomato soup, undiluted\nsalt\npepper\nsugar\nsalad oil\nvinegar\n\nTitle:\nMarinated Carrots\n",
            "Use:\nchocolate\nflour\nmilk\neggs\n\nTitle:\nchocolate cake\n",
      "Use:\ngal. cucumbers, peeled and cut\nlarge onions\ngarlic\nbell peppers\nsalt\nvinegar\nsugar\nturmeric\n\nTitle:\nCucumber Relish\n"]
for x in raw_text:
    prompt = tokenizer.encode(x , return_tensors = "pt")
    prompt = prompt.to(device)
    output = model.generate(prompt,do_sample = True, max_length = 200,top_k = 10, temperature = 0.8)
    output_text.append(tokenizer.decode(output[0] , skip_special_tokens = True))
for t in output_text:
  print(t)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Use:
carrots, cut in diagonal large onion, sliced in rings
tomato soup, undiluted
salt
pepper
sugar
salad oil
vinegar

Title:
Marinated Carrots

Ingredients:
2 lb. carrots, cut in diagonal slices
1 large onion, sliced in rings
1 can tomato soup, undiluted
1 tsp. salt
1 tsp. pepper
1 c. sugar
1/2 c. salad oil
1/2 c. vinegar

Directions:
cook carrots until tender-crisp.
drain.
mix onion rings and carrots with other ingredients.
refrigerate overnight.
Use:
chocolate
flour
milk
eggs

Title:
chocolate cake

Ingredients:
1/2 can chocolate
2 c. flour
1 pkg. milk
2 eggs

Directions:
mix chocolate and flour; add milk.
add eggs.
put in a 9 x 13-inch pan.
bake at 350° for 20 to 25 minutes.
Use:
gal. cucumbers, peeled and cut
large onions
garlic
bell peppers
salt
vinegar
sugar
turmeric

Title:
Cucumber Relish

Ingredients:
1 gal. cucumbers, peeled and cut
2 large onions
4 cloves garlic
2 bell peppers
1/4 c. salt
4 c. vinegar
8 c. sugar
1 tsp. turmeric

Directions:
put all ingredients in a large ke

In [ ]:
%ls
%cd ..
%ls

all.csv  all_processed.csv  lacto_ovo_processed.csv  vegan_processed.csv
/gdrive/My Drive/ai-finalproject
cache/   FinalModel/  results/       test0506_2.pt  test0514.pt    test051901.pt
data_2/  logs/        test0506_1.pt  test051402.pt  test051801.pt


In [ ]:
import torch
from transformers import  GPT2LMHeadModel, GPT2Tokenizer,AdamW
output_path = "FinalModel/vegan.pt"
model_path = output_path
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# Set the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
import nltk
from tqdm import tqdm
nltk.download('punkt')

file_path = 'data_2/all_processed.csv'
df = pd.read_csv(file_path, nrows = 6000*18)

tmp = df['full_recipe'].tolist()[6000*16:6000*16+1000]
candidate_texts = df['full_recipe'].tolist()[6000*16:6000*16+1000]
reference_texts = []
for x in tqdm(tmp):
    x = x.split("Ingredients:")[0]
    prompt = tokenizer.encode(x , return_tensors = "pt",max_length = 100 )
    prompt = prompt.to(device)
    output = model.generate(prompt,do_sample = True, max_length = 200,top_k = 10, temperature = 0.8)
    reference_texts.append(tokenizer.decode(output[0] , skip_special_tokens = True))

f1_scores = []

for ref_text, can_text in tqdm(zip(reference_texts, candidate_texts), total=len(reference_texts)):
    reference_tokens = nltk.word_tokenize(ref_text)
    candidate_tokens = nltk.word_tokenize(can_text)

    reference_labels = [1] * len(reference_tokens)
    candidate_labels = [1 if token in reference_tokens else 0 for token in candidate_tokens]

    min_length = min(len(reference_labels), len(candidate_labels))
    reference_labels = reference_labels[:min_length]
    candidate_labels = candidate_labels[:min_length]

    f1 = f1_score(reference_labels, candidate_labels)
    f1_scores.append(f1)

average_f1 = sum(f1_scores) / len(f1_scores)

print(f"\nAverage F1 Score: {average_f1:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  0%|          | 0/1000 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  0%|          | 1/1000 [00:17<4:59:07, 17.97s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  0%|          | 2/1000 [00:32<4:26:15, 16.01s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati


Average F1 Score: 0.8999


In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
import nltk
from tqdm import tqdm
nltk.download('punkt')

file_path = 'data_2/lacto_ovo_processed.csv'
df = pd.read_csv(file_path, nrows = 6000*18)

tmp = df['full_recipe'].tolist()[6000*16:6000*16+1000]
candidate_texts = df['full_recipe'].tolist()[6000*16:6000*16+1000]
reference_texts = []
for x in tqdm(tmp):
    x = x.split("Ingredients:")[0]
    prompt = tokenizer.encode(x , return_tensors = "pt",max_length = 100 )
    prompt = prompt.to(device)
    output = model.generate(prompt,do_sample = True, max_length = 200,top_k = 10, temperature = 0.8)
    reference_texts.append(tokenizer.decode(output[0] , skip_special_tokens = True))

f1_scores = []

for ref_text, can_text in tqdm(zip(reference_texts, candidate_texts), total=len(reference_texts)):
    reference_tokens = nltk.word_tokenize(ref_text)
    candidate_tokens = nltk.word_tokenize(can_text)

    reference_labels = [1] * len(reference_tokens)
    candidate_labels = [1 if token in reference_tokens else 0 for token in candidate_tokens]

    min_length = min(len(reference_labels), len(candidate_labels))
    reference_labels = reference_labels[:min_length]
    candidate_labels = candidate_labels[:min_length]

    f1 = f1_score(reference_labels, candidate_labels)
    f1_scores.append(f1)

average_f1 = sum(f1_scores) / len(f1_scores)

print(f"\nAverage F1 Score: {average_f1:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
  0%|          | 0/1000 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  0%|          | 1/1000 [00:19<5:28:10, 19.71s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
S


Average F1 Score: 0.8951


In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
import nltk
from tqdm import tqdm
nltk.download('punkt')

file_path = 'data_2/vegan_processed.csv'
df = pd.read_csv(file_path, nrows = 6000*18)

tmp = df['full_recipe'].tolist()[6000*16:6000*16+1000]
candidate_texts = df['full_recipe'].tolist()[6000*16:6000*16+1000]
reference_texts = []
for x in tqdm(tmp):

    x = x.split("Ingredients:")[0]
    prompt = tokenizer.encode(x , return_tensors = "pt",max_length = 100 )
    prompt = prompt.to(device)
    output = model.generate(prompt,do_sample = True, max_length = 200,top_k = 10, temperature = 0.8)
    reference_texts.append(tokenizer.decode(output[0] , skip_special_tokens = True))

f1_scores = []

for ref_text, can_text in tqdm(zip(reference_texts, candidate_texts), total=len(reference_texts)):
    reference_tokens = nltk.word_tokenize(ref_text)
    candidate_tokens = nltk.word_tokenize(can_text)

    reference_labels = [1] * len(reference_tokens)
    candidate_labels = [1 if token in reference_tokens else 0 for token in candidate_tokens]

    min_length = min(len(reference_labels), len(candidate_labels))
    reference_labels = reference_labels[:min_length]
    candidate_labels = candidate_labels[:min_length]

    f1 = f1_score(reference_labels, candidate_labels)
    f1_scores.append(f1)

average_f1 = sum(f1_scores) / len(f1_scores)

print(f"\nAverage F1 Score: {average_f1:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  0%|          | 0/1000 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  0%|          | 1/1000 [00:08<2:28:09,  8.90s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul


Average F1 Score: 0.8932


In [ ]:
print(reference_texts)

['Use:\nlight brown sugar\nmargarine, melted\neggs, beaten light\ncornmeal\nwater\nvanilla\nsalt\npecans, chopped\nunbaked pie shell\n\nTitle:\nPecan Pie\n\n\nIngredients:\n1 c. light brown sugar\n1/4 c. margarine, melted\n2 eggs, beaten (4)\n1/2 c. light\n1/2 c. cornmeal\n1/2 c. water\n1 tsp. vanilla\n1/4 tsp. salt\n1 c. pecans, chopped\n1 unbaked pie shell\n\nDirections:\nin a mixing bowl, combine brown sugar, margarine, sugar, eggs and water.\nstir in vanilla, salt and nuts.\npour into pie shell.\nbake for 30 minutes at 350°.\nreduce heat to 350° and bake for 30 minutes more.\nmakes one 9-inch pie.']


ignore below

In [ ]:
import torch

modell = Model.load_from_checkpoint("test0506_2.pt")
gpt2.state_dict = modell.state_dict
raw_text = ["Ingredients: milk. Title:" ,"Ingredients: 1 box powdered sugar 8 oz. Title:"  , "Ingredients: 1 box yellow cake mix 2 eggs 1/3 c. soft margarine 1 can Eagle Brand milk 1 c. pecans 1/2 c. Bits 'O Brickle. Title:" , "Ingredients: 1 can Spanish rice 1 can corned beef 1 diced onion 1/2 c. grated Cheddar cheese. Title:",
            "Ingredients: 3 1/2 c. flour\n3 c. sugar\n2 tsp. baking soda\n1 1/2 tsp. salt\n1 tsp. cinnamon\n1 tsp. nutmeg\n1 c. oil\n2/3 c. water\n4 eggs\n2 c. canned pumpkin.\nTitle: Pumpkin Bread\nReceipe: "]
output_text = []

gpt2.to("cpu")
for x in raw_text:
    prompt = tokenizer.encode(x , return_tensors = "pt")
    prompt.to("cpu")
    output = gpt2.generate(prompt,do_sample = True, max_length = 300,top_k = 10, temperature = 0.8)
    output_text.append(tokenizer.decode(output[0] , skip_special_tokens = True))

for t in output_text:
  print(t)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Ingredients: milk. Title: Creamy Coffee Cake.
Receipe: Mix sugar.
Beat eggs and add milk.
Beat until smooth.
Add milk gradually, mixing with fork.
Pour into baked sifted cake. Refrigerate.
Makes 2.
Ingredients: 1 box powdered sugar 8 oz. Title: Fudge Pie.
Receipe: Mix sugar and beat until smooth.
Beat until smooth and light.
Add sugar and beat until mixture is smooth and creamy.
Form into ball and put into pitcher.
Set in refrigerator until balls get hard and broken in half.
Ingredients: 1 box yellow cake mix 2 eggs 1/3 c. soft margarine 1 can Eagle Brand milk 1 c. pecans 1/2 c. Bits 'O Brickle. Title: Brickle Bars.
Receipe: Mix 2 eggs.
Add 1/3 cup margarine. Mix 2 more cups of flour and 1/2 cup of sugar; pour over the other ingredients.
Bake at 350\u00b0 for 25 to 35 minutes in a 9 x 12-inch pan, uncovered.
Makes about 12 servings, spread over the cake.
Ingredients: 1 can Spanish rice 1 can corned beef 1 diced onion 1/2 c. grated Cheddar cheese. Title: Cheddar And Cheddar Casserole.
R

test

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

%cd '/gdrive/My Drive/ai-finalproject'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/ai-finalproject


In [ ]:
import torch
# import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

gpt2 = GPT2LMHeadModel.from_pretrained("gpt2", cache_dir="./cache/gpt2model")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", cache_dir="./cache/gpt2token")
tokenizer.pad_token = tokenizer.eos_token

gpt2.load_state_dict(torch.load('test0506_2.pt'))
model.eval()

raw_text = ["Ingredients: milk. Title:" ,"Ingredients: 1 box powdered sugar 8 oz. Title:"  , "Ingredients: 1 box yellow cake mix 2 eggs 1/3 c. soft margarine 1 can Eagle Brand milk 1 c. pecans 1/2 c. Bits 'O Brickle. Title:" , "Ingredients: 1 can Spanish rice 1 can corned beef 1 diced onion 1/2 c. grated Cheddar cheese. Title:",
            "Ingredients: 3 1/2 c. flour\n3 c. sugar\n2 tsp. baking soda\n1 1/2 tsp. salt\n1 tsp. cinnamon\n1 tsp. nutmeg\n1 c. oil\n2/3 c. water\n4 eggs\n2 c. canned pumpkin.\nTitle: Pumpkin Bread\nReceipe: "]
output_text = []

gpt2.to("cpu")
for x in raw_text:
    prompt = tokenizer.encode(x , return_tensors = "pt")
    prompt.to("cpu")
    output = gpt2.generate(prompt,do_sample = True, max_length = 300,top_k = 10, temperature = 0.8)
    output_text.append(tokenizer.decode(output[0] , skip_special_tokens = True))

for t in output_text:
  print(t)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

RuntimeError: Error(s) in loading state_dict for GPT2LMHeadModel:
	Missing key(s) in state_dict: "transformer.wte.weight", "transformer.wpe.weight", "transformer.h.0.ln_1.weight", "transformer.h.0.ln_1.bias", "transformer.h.0.attn.c_attn.weight", "transformer.h.0.attn.c_attn.bias", "transformer.h.0.attn.c_proj.weight", "transformer.h.0.attn.c_proj.bias", "transformer.h.0.ln_2.weight", "transformer.h.0.ln_2.bias", "transformer.h.0.mlp.c_fc.weight", "transformer.h.0.mlp.c_fc.bias", "transformer.h.0.mlp.c_proj.weight", "transformer.h.0.mlp.c_proj.bias", "transformer.h.1.ln_1.weight", "transformer.h.1.ln_1.bias", "transformer.h.1.attn.c_attn.weight", "transformer.h.1.attn.c_attn.bias", "transformer.h.1.attn.c_proj.weight", "transformer.h.1.attn.c_proj.bias", "transformer.h.1.ln_2.weight", "transformer.h.1.ln_2.bias", "transformer.h.1.mlp.c_fc.weight", "transformer.h.1.mlp.c_fc.bias", "transformer.h.1.mlp.c_proj.weight", "transformer.h.1.mlp.c_proj.bias", "transformer.h.2.ln_1.weight", "tra

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd
from sklearn.metrics import f1_score
from tqdm import tqdm
nltk.download('punkt')

file_path = 'data_2/vegan_processed.csv'
df = pd.read_csv(file_path, nrows = 6000*18)

tmp = df['full_recipe'].tolist()[6000*16:6000*16+10]
candidate_texts = df['full_recipe'].tolist()[6000*16:6000*16+10]
reference_texts = []
for x in tqdm(tmp):
    x = x.split("Ingredients:")[0]
    prompt = tokenizer.encode(x , return_tensors = "pt",max_length = 100 )
    prompt = prompt.to(device)
    output = model.generate(prompt,do_sample = True, max_length = 200,top_k = 10, temperature = 0.8)
    reference_texts.append(tokenizer.decode(output[0] , skip_special_tokens = True))

bleu_scores = []

for ref_text, can_text in tqdm(zip(reference_texts, candidate_texts), total=len(reference_texts)):
    generated_tokens = ref_text.split()
    reference_tokens = [can_text.split()]



    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothie)

    bleu_scores.append(bleu_score)

average_bleu_scores = sum(bleu_scores) / len(bleu_scores)



print(f"BLEU score: {average_bleu_scores:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 10%|█         | 1/10 [00:06<00:56,  6.30s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 20%|██        | 2/10 [00:15<01:02,  7.84s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 30%|█

BLEU score: 0.3787


In [ ]:
import torch
from transformers import  GPT2LMHeadModel, GPT2Tokenizer,AdamW
output_path = "FinalModel/all.pt"
model_path = output_path
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# Set the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd
from sklearn.metrics import f1_score
from tqdm import tqdm
nltk.download('punkt')

file_path = 'data_2/all_processed.csv'
df = pd.read_csv(file_path, nrows = 6000*18)

tmp = df['full_recipe'].tolist()[6000*16:6000*16+10]
candidate_texts = df['full_recipe'].tolist()[6000*16:6000*16+10]
reference_texts = []
for x in tqdm(tmp):
    x = x.split("Ingredients:")[0]
    prompt = tokenizer.encode(x , return_tensors = "pt",max_length = 100 )
    prompt = prompt.to(device)
    output = model.generate(prompt,do_sample = True, max_length = 200,top_k = 10, temperature = 0.8)
    reference_texts.append(tokenizer.decode(output[0] , skip_special_tokens = True))

bleu_scores = []

for ref_text, can_text in tqdm(zip(reference_texts, candidate_texts), total=len(reference_texts)):
    generated_tokens = ref_text.split()
    reference_tokens = [can_text.split()]


    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothie)

    bleu_scores.append(bleu_score)

average_bleu_scores = sum(bleu_scores) / len(bleu_scores)



print(f"BLEU score: {average_bleu_scores:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  0%|          | 0/10 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 10%|█         | 1/10 [00:06<01:00,  6.76s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Se

BLEU score: 0.3978


In [ ]:
import torch
from transformers import  GPT2LMHeadModel, GPT2Tokenizer,AdamW
output_path = "FinalModel/lacto_ovo.pt"
model_path = output_path
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import pandas as pd
from sklearn.metrics import f1_score
from tqdm import tqdm

nltk.download('punkt')

file_path = 'data_2/lacto_ovo_processed.csv'
df = pd.read_csv(file_path, nrows = 6000*18)

tmp = df['full_recipe'].tolist()[6000*16:6000*16+10]
candidate_texts = df['full_recipe'].tolist()[6000*16:6000*16+10]
reference_texts = []
for x in tqdm(tmp):

    x = x.split("Ingredients:")[0]
    prompt = tokenizer.encode(x , return_tensors = "pt",max_length = 100 )
    prompt = prompt.to(device)
    output = model.generate(prompt,do_sample = True, max_length = 200,top_k = 10, temperature = 0.8)
    reference_texts.append(tokenizer.decode(output[0] , skip_special_tokens = True))

bleu_scores = []

for ref_text, can_text in tqdm(zip(reference_texts, candidate_texts), total=len(reference_texts)):
    generated_tokens = ref_text.split()
    reference_tokens = [can_text.split()]

    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothie)

    bleu_scores.append(bleu_score)

average_bleu_scores = sum(bleu_scores) / len(bleu_scores)



print(f"BLEU score: {average_bleu_scores:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
  0%|          | 0/10 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 10%|█         | 1/10 [00:10<01:36, 10.72s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Se

BLEU score: 0.4192


In [ ]:
print(reference_texts)
print(candidate_texts)

['Use:\nlight brown sugar\nmargarine, melted\neggs, beaten light\ncornmeal\nwater\nvanilla\nsalt\npecans, chopped\nunbaked pie shell\n\nTitle:\nPecan Pie\n\n\nIngredients:\n2 c. light brown sugar\n1 stick margarine, melted\n2 c. eggs, beaten light\n3/4 c. cornmeal\n1 1/2 c. water\n1 tsp. vanilla\n1/4 tsp. salt\n3/4 c. pecans, chopped\n1 unbaked pie shell\n\nDirections:\nmix the brown sugar, margarine, eggs, cornmeal, water, vanilla and salt in a large bowl.\npour in pie shell and bake at 350° for 40 to 45 minutes.\ntop with pecans and bake 15 to 20 more minutes.']
['Use:\nlight brown sugar\nmargarine, melted\neggs, beaten light\ncornmeal\nwater\nvanilla\nsalt\npecans, chopped\nunbaked pie shell\n\nTitle:\nPecan Pie\n\nIngredients:\n1 box light brown sugar\n1 stick margarine, melted\n4 eggs, beaten light\n2 tbsp. cornmeal\n2 tbsp. water\n1 tbsp. vanilla\npinch of salt\n2 c. pecans, chopped\n2 unbaked pie shell\n\nDirections:\nmix all ingredients together, pour into pie shells and bake f